In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder # pour l'encodage des variables
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import fanalysis.mca as mca
df=pd.read_csv("Train.csv")



In [2]:
df_sub=df.head(200)

In [3]:
df_sub.shape

(200, 4043)

In [4]:
df_sub.to_csv("sub.csv",index=False)

In [5]:
df[df['Accès internet'].isna()]
# On remarque la dernière ligne contient les valeurs manquantes presque pour toutes les attributs donc on va supprimer cette ligne
df=df.drop(14386)
df.describe()

,TAILLE_MENAGE,,.1,.2,.3,.4,.5,.6,.7,.8,...,.3991,.3992,.3993,.3994,.3995,.3996,.3997,.3998,.3999,Accès internet
count,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,...,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000,14386.000000
mean,4.458849,0.205889,0.528303,0.079629,0.315085,0.333881,0.478260,0.147020,0.251050,0.246089,...,0.179247,0.588116,0.370216,0.273030,0.963948,2.017242,0.663119,0.666739,0.107387,0.494856
std,2.926586,0.132027,0.296896,0.068591,0.212374,0.208956,0.287464,0.100793,0.172577,0.181236,...,0.171899,0.400228,0.257743,0.209297,0.598916,1.089832,0.438795,0.425746,0.104803,0.499991
min,1.000000,0.003657,0.013529,0.004931,0.000000,0.003523,0.024215,0.030554,0.020446,0.000000,...,0.003326,0.055927,0.010327,0.000387,0.004233,0.201520,0.010988,0.000710,0.005095,0.000000
25%,2.000000,0.115180,0.364513,0.038801,0.161350,0.209651,0.322518,0.081526,0.110521,0.098520,...,0.058396,0.320013,0.221761,0.137931,0.546373,1.201318,0.371847,0.388778,0.037873,0.000000
50%,4.000000,0.179487,0.532363,0.048690,0.333702,0.321623,0.414460,0.101029,0.168441,0.242293,...,0.103019,0.443965,0.293205,0.228299,1.012230,1.743152,0.652090,0.728096,0.056223,0.000000
75%,6.000000,0.266084,0.701194,0.091743,0.452244,0.454778,0.580604,0.171441,0.358821,0.357327,...,0.261872,0.654009,0.425018,0.396811,1.281234,2.344100,0.877277,0.864186,0.156302,1.000000
max,36.000000,0.714337,1.435041,0.410256,0.937503,1.010105,1.664593,0.637903,0.978567,0.811333,...,1.007384,2.376178,1.578969,1.134671,2.834302,6.501053,2.294453,1.828806,0.747823,1.000000


In [6]:
var_cat=['TypeLogmt_1','TypeLogmt_2','TypeLogmt_3','H08_Impute','H09_Impute','TAILLE_MENAGE','H17A','H17B','H17C','H17D','H17E','H17F','H17G','H17H','H17I','H17J','H17Y','H18A','H18B','H18C','H18D','H18E','H18F','H18G','H18H','H18I','H18J','H18Y','H20A','H20B','H20C','H20D','H20E','H20Y','H21A','H21B','H21C','H21D','H21Y']
var_pca_df = df.drop(['ID', 'Accès internet','Connexion','BoxLabel'] + var_cat, axis=1)
var_cat_df = df[var_cat]

In [7]:
var_cat_df.shape
var_pca_df.shape
var_pca_df.head()

,,.1,.2,.3,.4,.5,.6,.7,.8,.9,...,.3990,.3991,.3992,.3993,.3994,.3995,.3996,.3997,.3998,.3999
0,0.110878,0.364513,0.082156,0.050989,0.180205,0.440745,0.091101,0.350169,0.012656,0.414666,...,0.183824,0.025202,0.235426,0.251377,0.125899,0.834979,1.055083,0.503122,0.761878,0.018388
1,0.070467,0.174473,0.077245,0.009509,0.065423,0.181117,0.089368,0.513371,0.007979,0.136733,...,0.216348,0.417944,0.395539,0.243477,0.408191,0.278853,1.134487,0.165434,0.117637,0.214102
2,0.184448,0.529424,0.038106,0.408890,0.344663,0.368679,0.098398,0.116663,0.328337,0.416548,...,0.280863,0.059764,0.368242,0.255468,0.195145,0.976152,1.570176,0.669818,0.736460,0.037873
3,0.173134,0.488908,0.044793,0.299539,0.313499,0.426035,0.083545,0.120959,0.212011,0.442777,...,0.335834,0.101552,0.474090,0.342558,0.228299,1.054081,1.805052,0.603918,0.746610,0.051638
4,0.166340,0.532463,0.034355,0.400996,0.301856,0.404993,0.072722,0.101579,0.324310,0.436339,...,0.318358,0.094498,0.410741,0.248533,0.222522,1.012230,1.725041,0.582654,0.770315,0.037564


In [8]:
scaler = StandardScaler()
var_pca_nor = scaler.fit_transform(var_pca_df)

In [9]:
pca = PCA(n_components=350)
var_pca_nor_red = pca.fit_transform(var_pca_nor)


In [10]:
X = pd.concat([var_cat_df, pd.DataFrame(var_pca_nor_red)], axis=1)
scaler_pca = StandardScaler()

In [11]:
X.head()
#X.describe()


,TypeLogmt_1,TypeLogmt_2,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,...,340,341,342,343,344,345,346,347,348,349
0,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,1.0,Non,Oui,Non,Non,...,-0.000896,0.001010,0.001007,0.000299,0.000555,-0.000403,-0.000062,0.000232,0.000795,-0.000249
1,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,4.0,Non,Oui,Non,Non,...,-0.000278,0.000019,0.000421,0.000122,0.000359,-0.000052,-0.000091,-0.000173,0.000005,-0.000318
2,Logement moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,1.0,Non,Non,Non,Non,...,0.005624,-0.000408,0.000664,0.003585,0.002287,0.001732,0.001148,-0.002147,0.001233,-0.002059
3,Logement moderne,Plusieurs logement,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,4.0,Non,Non,Non,Non,...,-0.004885,-0.004612,-0.000686,0.001838,-0.004310,-0.000489,-0.003028,0.001493,0.000316,-0.000125
4,Logement moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,1.0,Non,Non,Non,Non,...,-0.001875,-0.000394,-0.000871,-0.000352,-0.000121,-0.000992,0.000459,-0.000484,0.000590,0.000198


In [12]:
from scipy.stats import chi2_contingency

# Initialiser les DataFrame pour les coefficients de Cramér et les p-values
cramer_v_df = pd.DataFrame(index=var_cat, columns=var_cat)
p_value_df = pd.DataFrame(index=var_cat, columns=var_cat)
tschuprow_t_df = pd.DataFrame(index=var_cat, columns=var_cat)


# Calculer le test de chi-deux pour chaque paire de variables qualitatives
for i, column1 in enumerate(var_cat):
    for j, column2 in enumerate(var_cat):
        if column1 != column2:
            contingency_table = pd.crosstab(df[column1], df[column2])
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            cramer_v = np.sqrt(chi2 / (df.shape[0] * (min(contingency_table.shape) - 1)))
            tschuprow_t = cramer_v * np.sqrt((contingency_table.shape[0] - 1) * (contingency_table.shape[1] - 1) / (df.shape[0] - 1))
            cramer_v_df.loc[column1, column2] = cramer_v
            tschuprow_t_df.loc[column1, column2] = tschuprow_t
            p_value_df.loc[column1, column2] = p


In [13]:
# Afficher la DataFrame des coefficients de Cramér
print("DataFrame des coefficients de Cramér :")
cramer_v_df


DataFrame des coefficients de Cramér :


,TypeLogmt_1,TypeLogmt_2,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,...,H20B,H20C,H20D,H20E,H20Y,H21A,H21B,H21C,H21D,H21Y
TypeLogmt_1,NaN,0.714943,0.603313,0.158511,0.177263,0.051263,0.037682,0.094141,0.019404,0.02235,...,0.048242,0.279245,0.044466,0.146666,0.166621,0.009273,0.018481,0.037662,0.114212,0.106895
TypeLogmt_2,0.714943,NaN,0.73005,0.066506,0.084847,0.049984,0.01566,0.02309,0.013295,0.02766,...,0.013878,0.043512,0.013167,0.019902,0.038487,0.01459,0.008885,0.008998,0.044199,0.031042
TypeLogmt_3,0.603313,0.73005,NaN,0.505434,0.183378,0.05892,0.018107,0.025653,0.017891,0.027169,...,0.02445,0.186341,0.061148,0.10377,0.09761,0.016628,0.009665,0.040316,0.143262,0.108091
H08_Impute,0.158511,0.066506,0.505434,NaN,0.12909,0.09892,0.108544,0.107227,0.059015,0.03079,...,0.111506,0.220401,0.056177,0.116724,0.270231,0.016835,0.014539,0.07352,0.087853,0.04936
H09_Impute,0.177263,0.084847,0.183378,0.12909,NaN,0.085188,0.091071,0.072882,0.055423,0.050377,...,0.088246,0.259154,0.091543,0.165614,0.199756,0.029844,0.098329,0.101334,0.126452,0.101912
TAILLE_MENAGE,0.051263,0.049984,0.05892,0.09892,0.085188,NaN,0.187013,0.2567,0.050627,0.065115,...,0.087297,0.150306,0.047314,0.116937,0.164597,0.042346,0.029232,0.096281,0.070495,0.096813
H17A,0.037682,0.01566,0.018107,0.108544,0.091071,0.187013,NaN,0.1276,0.006483,0.040924,...,0.017821,0.059029,0.031635,0.080827,0.05257,0.022069,0.004411,0.051579,0.054434,0.073359
H17B,0.094141,0.02309,0.025653,0.107227,0.072882,0.2567,0.1276,NaN,0.062678,0.016491,...,0.078119,0.197812,0.037005,0.143585,0.217792,0.018697,0.0,0.052354,0.056895,0.078447
H17C,0.019404,0.013295,0.017891,0.059015,0.055423,0.050627,0.006483,0.062678,NaN,0.044067,...,0.0362,0.009835,0.015925,0.002405,0.013514,0.024249,0.011023,0.02496,0.0,0.019307
H17D,0.02235,0.02766,0.027169,0.03079,0.050377,0.065115,0.040924,0.016491,0.044067,NaN,...,0.025841,0.036901,0.050789,0.026185,0.034752,0.032275,0.038104,0.01522,0.016758,0.026801


In [14]:
X=X.drop(['TypeLogmt_1','TypeLogmt_2','H21C','H21D'],axis=1)


In [15]:
X.head()

,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,...,340,341,342,343,344,345,346,347,348,349
0,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,1.0,Non,Oui,Non,Non,Non,Non,...,-0.000896,0.001010,0.001007,0.000299,0.000555,-0.000403,-0.000062,0.000232,0.000795,-0.000249
1,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,4.0,Non,Oui,Non,Non,Non,Non,...,-0.000278,0.000019,0.000421,0.000122,0.000359,-0.000052,-0.000091,-0.000173,0.000005,-0.000318
2,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,1.0,Non,Non,Non,Non,Non,Non,...,0.005624,-0.000408,0.000664,0.003585,0.002287,0.001732,0.001148,-0.002147,0.001233,-0.002059
3,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,4.0,Non,Non,Non,Non,Non,Non,...,-0.004885,-0.004612,-0.000686,0.001838,-0.004310,-0.000489,-0.003028,0.001493,0.000316,-0.000125
4,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,1.0,Non,Non,Non,Non,Non,Non,...,-0.001875,-0.000394,-0.000871,-0.000352,-0.000121,-0.000992,0.000459,-0.000484,0.000590,0.000198


In [16]:

"""X_cat = df[var_cat].values
# Créer une instance de la classe MCA
my_mca = mca.MCA()

my_mca.fit(X_cat)
print(my_mca.eig_)"""


'X_cat = df[var_cat].values\n# Créer une instance de la classe MCA\nmy_mca = mca.MCA()\n\nmy_mca.fit(X_cat)\nprint(my_mca.eig_)'

In [17]:
#my_mca.plot_eigenvalues()

In [18]:
#my_mca.plot_eigenvalues(type="cumulative")

In [19]:
"""scaler_taille_menage = StandardScaler()
X['TAILLE_MENAGE']= scaler_taille_menage.fit_transform(X[['TAILLE_MENAGE']])"""


In [20]:
"""listed_de_variable_object_contenant_oui_non=['H17A','H17B','H17C','H17D','H17E','H17F','H17G','H17H','H17I','H17J','H17Y','H18A','H18B','H18C','H18D','H18E','H18F','H18G','H18H','H18I','H18J','H18Y','H20A','H20B','H20C','H20D','H20E','H20Y','H21A','H21B','H21Y']
len(listed_de_variable_object_contenant_oui_non)
for col in listed_de_variable_object_contenant_oui_non:
    # Utilisez une condition pour remplacer les valeurs "oui" par 1 et les valeurs "non" par 0
    X[col] = X[col].apply(lambda x: 1 if x == "Oui" else 0)"""


In [21]:
X.head()

,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,...,340,341,342,343,344,345,346,347,348,349
0,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,-1.181913,0,1,0,0,0,0,...,-0.000896,0.001010,0.001007,0.000299,0.000555,-0.000403,-0.000062,0.000232,0.000795,-0.000249
1,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,-0.156792,0,1,0,0,0,0,...,-0.000278,0.000019,0.000421,0.000122,0.000359,-0.000052,-0.000091,-0.000173,0.000005,-0.000318
2,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,-1.181913,0,0,0,0,0,0,...,0.005624,-0.000408,0.000664,0.003585,0.002287,0.001732,0.001148,-0.002147,0.001233,-0.002059
3,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,-0.156792,0,0,0,0,0,0,...,-0.004885,-0.004612,-0.000686,0.001838,-0.004310,-0.000489,-0.003028,0.001493,0.000316,-0.000125
4,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,-1.181913,0,0,0,0,0,0,...,-0.001875,-0.000394,-0.000871,-0.000352,-0.000121,-0.000992,0.000459,-0.000484,0.000590,0.000198


In [22]:
liste_type_logement=['TypeLogmt_3']
for col in liste_type_logement:
    valeurs_uniques = X[col].unique()
    print(valeurs_uniques)


['Logement � un niveau (plain-pied)' 'Logement ? un niveau (plain-pied)'
 'Logement � plusieurs niveaux (� �tage)'
 'Logement ? plusieurs niveaux (? ?tage)' 'Autre']


In [23]:
X['TypeLogmt_3'] = X['TypeLogmt_3'].replace({
    'Logement � un niveau (plain-pied)': 'Logement a un niveau (plain-pied)',
    'Logement ? un niveau (plain-pied)': 'Logement a un niveau (plain-pied)',
    'Logement ? plusieurs niveaux (? ?tage)': 'Logement a plusieurs niveaux (a etage)',
    'Logement � plusieurs niveaux (� �tage)':'Logement a plusieurs niveaux (a etage)'
})

In [24]:
print(X['TypeLogmt_3'].unique())

['Logement a un niveau (plain-pied)'
 'Logement a plusieurs niveaux (a etage)' 'Autre']


In [25]:
X['H08_Impute'] = X['H08_Impute'].replace({
    'Electricit� (CEET) compteur dans la concession':'electricite comp in maison',
    'Electricit? (CEET) compteur dans la concession': 'electricite comp in maison',
    'Electricit? (CEET) compteur hors concession':'electricite comp hors maison',
    'Electricit� (CEET) compteur hors concession':'electricite comp hors maison',
    'C�blage d��lectricit� du voisinnage':'electricite comp hors maison',
    'C?blage d??lectricit? du voisinnage': 'electricite comp hors maison',
    'Lampion': 'Lampion',
    'Torche/bougie': 'torche',
    'Lampe � p�trole': 'Lampe petrole',
    'Lampe ? p?trole': 'Lampe petrole',
    'Lampe ? gaz': 'Lampe a gaz',
    'Lampe � gaz': 'Lampe a gaz',
    'Groupe �lectrog�ne': 'groupe',
    'Groupe ?lectrog?ne': 'groupe',
    'Autre (� pr�ciser)': 'autres',
    'Autre (? pr?ciser)': 'autres'
    })


In [26]:
print(X['H09_Impute'].unique())

['Gaz' 'Charbon de bois' 'Ne cuisine pas' 'Bois de chauffe' '�lectricit�'
 'P�trole' '?lectricit?' 'P?trole' 'Autre (� pr�ciser)'
 'R�sidus v�g�taux/sciure de bois' 'Autre (? pr?ciser)'
 'R?sidus v?g?taux/sciure de bois' 'Bouse d�animaux']


In [27]:
X['H09_Impute'] = X['H09_Impute'].replace({
    '�lectricit�':'electricite',
    '?lectricit?':'electricite',
    'P�trole':'petrole',
    'P?trole':'petrole',
    'Autre (� pr�ciser)':'autres',
    'R�sidus v�g�taux/sciure de bois':'traditionnel',
    'Autre (? pr?ciser)':'autres',
    'R?sidus v?g?taux/sciure de bois':'traditionnel',
    'Bouse d�animaux':'traditionnel',
    'Bois de chauffe':'traditionnel'
    

    })

In [28]:
"""colonnes_cat= ["TypeLogmt_3","H08_Impute","H09_Impute"]
X = pd.get_dummies(X, columns=colonnes_cat,dtype=float)"""

In [29]:
X.head()

,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,...,H08_Impute_electricite comp in maison,H08_Impute_groupe,H08_Impute_torche,H09_Impute_Charbon de bois,H09_Impute_Gaz,H09_Impute_Ne cuisine pas,H09_Impute_autres,H09_Impute_electricite,H09_Impute_petrole,H09_Impute_traditionnel
0,-1.181913,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.156792,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-1.181913,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.156792,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-1.181913,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Convertir les valeurs True et False en 1 et 0 respectivement


In [31]:
X.head()


,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,...,H08_Impute_electricite comp in maison,H08_Impute_groupe,H08_Impute_torche,H09_Impute_Charbon de bois,H09_Impute_Gaz,H09_Impute_Ne cuisine pas,H09_Impute_autres,H09_Impute_electricite,H09_Impute_petrole,H09_Impute_traditionnel
0,-1.181913,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.156792,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-1.181913,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.156792,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-1.181913,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X.columns = X.columns.astype(str)
y=df['Accès internet']

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape
X_train.head()

,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,...,H08_Impute_electricite comp in maison,H08_Impute_groupe,H08_Impute_torche,H09_Impute_Charbon de bois,H09_Impute_Gaz,H09_Impute_Ne cuisine pas,H09_Impute_autres,H09_Impute_electricite,H09_Impute_petrole,H09_Impute_traditionnel
11481,-0.498499,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
13614,-0.840206,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8107,0.184915,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8176,0.526622,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9292,0.184915,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Initialiser le modèle XGBoost
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Définir une grille de recherche pour les hyperparamètres
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Configurer GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# Effectuer la recherche des meilleurs hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print('Best parameters found: ', grid_search.best_params_)

# Meilleur modèle
best_xgb = grid_search.best_estimator_


Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters found:  {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 300, 'subsample': 0.8}


In [36]:
# Prédictions sur les données de test
y_pred = best_xgb.predict(X_val)
y_pred_proba = best_xgb.predict_proba(X_val)[:, 1]

# Évaluation des performances
print(classification_report(y_val, y_pred))
print('AUC-ROC:', roc_auc_score(y_val, y_pred_proba))

              precision    recall  f1-score   support

         0.0       0.86      0.75      0.80      1466
         1.0       0.77      0.87      0.82      1412

    accuracy                           0.81      2878
   macro avg       0.81      0.81      0.81      2878
weighted avg       0.81      0.81      0.81      2878

AUC-ROC: 0.8594187803624361


In [ ]:
"""# Entraînement d'un modèle RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Évaluation du modèle
y_pred = rf.predict(X_val)
#print(classification_report(y_val, y_pred))
print('AUC-ROC:', roc_auc_score(y_val, rf.predict_proba(X_val)[:, 1]))"""

In [ ]:
"""param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30]
}
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)
print('Best parameters:', grid_search.best_params_)"""

In [ ]:
#best_rf = grid_search.best_estimator_

In [ ]:
#print('AUC-ROC:', roc_auc_score(y_val, best_rf.predict_proba(X_val)[:, 1]))

In [ ]:
"""model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Calculez l'AUC
auc = roc_auc_score(y_val, y_pred_proba)
print("AUC:", auc)"""

In [37]:
df_test=pd.read_csv("Test.csv")

df_test.head()

,ID,TypeLogmt_1,TypeLogmt_2,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,...,.3990,.3991,.3992,.3993,.3994,.3995,.3996,.3997,.3998,.3999
0,ID_6R2X9XE3RA,Logement moderne,Logement unique,Logement ? plusieurs niveaux (? ?tage),Electricit? (CEET) compteur hors concession,Charbon de bois,5.0,Non,Oui,Non,...,0.215417,0.062298,0.328880,0.193603,0.123900,0.773756,1.464494,0.647847,0.448600,0.047404
1,ID_1CQDGXQJW9,Logement moderne,Plusieurs logement,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,9.0,Non,Oui,Non,...,0.023053,0.029340,0.204032,0.067190,0.004446,0.034146,0.841245,0.036763,0.003054,0.027603
2,ID_5DQVKM2ODB,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,1.0,Non,Non,Non,...,0.349223,0.096777,0.402451,0.234141,0.230451,1.077133,1.731721,0.600732,0.784672,0.037761
3,ID_65TQFKG2L2,Logement semi-moderne,Logement unique,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,3.0,Non,Non,Non,...,0.506753,0.151268,0.624592,0.360747,0.351187,1.323039,2.344100,0.900833,0.990587,0.085504
4,ID_NBG4Q4OAF2,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,3.0,Non,Non,Oui,...,0.401449,0.127706,0.516138,0.312065,0.277865,1.163575,1.976314,0.677894,0.851412,0.060335


In [ ]:
df_test.shape

In [38]:
df_test.isnull().sum()

ID             0
TypeLogmt_1    0
TypeLogmt_2    0
TypeLogmt_3    0
H08_Impute     0
              ..
 .3995         0
 .3996         0
 .3997         0
 .3998         0
 .3999         0
Length: 4042, dtype: int64

In [39]:
var_cat1=['TypeLogmt_1','TypeLogmt_2','TypeLogmt_3','H08_Impute','H09_Impute','TAILLE_MENAGE','H17A','H17B','H17C','H17D','H17E','H17F','H17G','H17H','H17I','H17J','H17Y','H18A','H18B','H18C','H18D','H18E','H18F','H18G','H18H','H18I','H18J','H18Y','H20A','H20B','H20C','H20D','H20E','H20Y','H21A','H21B','H21C','H21D','H21Y']
var_pca1_df = df_test.drop(['ID','Connexion','BoxLabel'] + var_cat1, axis=1)
var_cat1_df = df_test[var_cat1]

In [40]:
var_pca1_df.head()

,,.1,.2,.3,.4,.5,.6,.7,.8,.9,...,.3990,.3991,.3992,.3993,.3994,.3995,.3996,.3997,.3998,.3999
0,0.176887,0.469221,0.054311,0.250623,0.302368,0.398230,0.152434,0.147250,0.185170,0.379103,...,0.215417,0.062298,0.328880,0.193603,0.123900,0.773756,1.464494,0.647847,0.448600,0.047404
1,0.151004,0.477356,0.040247,0.298954,0.287559,0.392975,0.099656,0.130923,0.216809,0.429798,...,0.023053,0.029340,0.204032,0.067190,0.004446,0.034146,0.841245,0.036763,0.003054,0.027603
2,0.026003,0.028151,0.044258,0.000105,0.020683,0.091367,0.072772,0.426237,0.000009,0.061014,...,0.349223,0.096777,0.402451,0.234141,0.230451,1.077133,1.731721,0.600732,0.784672,0.037761
3,0.277901,0.701194,0.071182,0.518991,0.466243,0.526012,0.145066,0.168441,0.428502,0.563009,...,0.506753,0.151268,0.624592,0.360747,0.351187,1.323039,2.344100,0.900833,0.990587,0.085504
4,0.196737,0.597471,0.045454,0.441222,0.356589,0.473065,0.089168,0.126996,0.357327,0.486618,...,0.401449,0.127706,0.516138,0.312065,0.277865,1.163575,1.976314,0.677894,0.851412,0.060335


In [41]:
scaler1 = StandardScaler()
var_pca1_nor = scaler1.fit_transform(var_pca1_df)

In [42]:
pca1 = PCA(n_components=350)
var_pca1_nor_red = pca1.fit_transform(var_pca1_nor)


In [43]:
df_t= pd.concat([var_cat1_df, pd.DataFrame(var_pca1_nor_red)], axis=1)


In [44]:
df_t.head()

,TypeLogmt_1,TypeLogmt_2,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,...,340,341,342,343,344,345,346,347,348,349
0,Logement moderne,Logement unique,Logement ? plusieurs niveaux (? ?tage),Electricit? (CEET) compteur hors concession,Charbon de bois,5.0,Non,Oui,Non,Non,...,-0.006544,0.010855,0.004456,0.008563,0.011695,0.009527,-0.010718,-0.007084,0.001968,-0.005246
1,Logement moderne,Plusieurs logement,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,9.0,Non,Oui,Non,Non,...,-0.002221,0.002049,0.001740,0.002775,-0.002943,-0.005496,-0.001192,-0.004590,0.000159,0.001402
2,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,1.0,Non,Non,Non,Non,...,-0.001471,-0.001577,-0.001217,-0.000076,-0.001065,-0.000691,0.001445,0.000470,-0.001055,-0.000578
3,Logement semi-moderne,Logement unique,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,3.0,Non,Non,Non,Non,...,-0.000196,0.001167,-0.000002,-0.000764,-0.000267,-0.000513,-0.000335,-0.000083,-0.000238,0.000039
4,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,3.0,Non,Non,Oui,Non,...,-0.000477,0.000041,-0.000784,0.000196,-0.000150,-0.000341,0.000439,0.000737,0.000982,-0.000262


In [45]:
scaler_taille_menage1= StandardScaler()
df_t['TAILLE_MENAGE']= scaler_taille_menage1.fit_transform(df_t[['TAILLE_MENAGE']])

In [46]:
df_t=df_t.drop(['TypeLogmt_1','TypeLogmt_2','H21C','H21D'],axis=1)


In [47]:
len(listed_de_variable_object_contenant_oui_non)
for col in listed_de_variable_object_contenant_oui_non:
    # Utilisez une condition pour remplacer les valeurs "oui" par 1 et les valeurs "non" par 0
    df_t[col] = df_t[col].apply(lambda x: 1 if x == "Oui" else 0)


In [48]:
df_t.head()

,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,...,340,341,342,343,344,345,346,347,348,349
0,Logement ? plusieurs niveaux (? ?tage),Electricit? (CEET) compteur hors concession,Charbon de bois,0.162374,0,1,0,0,0,0,...,-0.006544,0.010855,0.004456,0.008563,0.011695,0.009527,-0.010718,-0.007084,0.001968,-0.005246
1,Logement ? un niveau (plain-pied),Electricit? (CEET) compteur dans la concession,Gaz,1.512391,0,1,0,0,0,0,...,-0.002221,0.002049,0.001740,0.002775,-0.002943,-0.005496,-0.001192,-0.004590,0.000159,0.001402
2,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,-1.187643,0,0,0,0,0,0,...,-0.001471,-0.001577,-0.001217,-0.000076,-0.001065,-0.000691,0.001445,0.000470,-0.001055,-0.000578
3,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Charbon de bois,-0.512634,0,0,0,0,0,0,...,-0.000196,0.001167,-0.000002,-0.000764,-0.000267,-0.000513,-0.000335,-0.000083,-0.000238,0.000039
4,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,-0.512634,0,0,1,0,0,0,...,-0.000477,0.000041,-0.000784,0.000196,-0.000150,-0.000341,0.000439,0.000737,0.000982,-0.000262


In [49]:
df_t['TypeLogmt_3'] = df_t['TypeLogmt_3'].replace({
    'Logement � un niveau (plain-pied)': 'Logement a un niveau (plain-pied)',
    'Logement ? un niveau (plain-pied)': 'Logement a un niveau (plain-pied)',
    'Logement ? plusieurs niveaux (? ?tage)': 'Logement a plusieurs niveaux (a etage)',
    'Logement � plusieurs niveaux (� �tage)':'Logement a plusieurs niveaux (a etage)'
})

In [50]:
df_t['H08_Impute'] = df_t['H08_Impute'].replace({
    'Electricit� (CEET) compteur dans la concession':'electricite comp in maison',
    'Electricit? (CEET) compteur dans la concession': 'electricite comp in maison',
    'Electricit? (CEET) compteur hors concession':'electricite comp hors maison',
    'Electricit� (CEET) compteur hors concession':'electricite comp hors maison',
    'C�blage d��lectricit� du voisinnage':'electricite comp hors maison',
    'C?blage d??lectricit? du voisinnage': 'electricite comp hors maison',
    'Lampion': 'Lampion',
    'Torche/bougie': 'torche',
    'Lampe � p�trole': 'Lampe petrole',
    'Lampe ? p?trole': 'Lampe petrole',
    'Lampe ? gaz': 'Lampe a gaz',
    'Lampe � gaz': 'Lampe a gaz',
    'Groupe �lectrog�ne': 'groupe',
    'Groupe ?lectrog?ne': 'groupe',
    'Autre (� pr�ciser)': 'autres',
    'Autre (? pr?ciser)': 'autres'
    })


In [51]:
df_t['H09_Impute'] = df_t['H09_Impute'].replace({
    '�lectricit�':'electricite',
    '?lectricit?':'electricite',
    'P�trole':'petrole',
    'P?trole':'petrole',
    'Autre (� pr�ciser)':'autres',
    'R�sidus v�g�taux/sciure de bois':'traditionnel',
    'Autre (? pr?ciser)':'autres',
    'R?sidus v?g?taux/sciure de bois':'traditionnel',
    'Bouse d�animaux':'traditionnel',
    'Bois de chauffe':'traditionnel'
    

    })


In [52]:
colonnes_cat1= ["TypeLogmt_3","H08_Impute","H09_Impute"]
df_t = pd.get_dummies(df_t, columns=colonnes_cat1,dtype=float)


In [53]:
df_t.head()

,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,...,H08_Impute_electricite comp in maison,H08_Impute_groupe,H08_Impute_torche,H09_Impute_Charbon de bois,H09_Impute_Gaz,H09_Impute_Ne cuisine pas,H09_Impute_autres,H09_Impute_electricite,H09_Impute_petrole,H09_Impute_traditionnel
0,0.162374,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.512391,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-1.187643,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.512634,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.512634,0,0,1,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df_t.columns = df_t.columns.astype(str)
submissions=best_xgb.predict(df_t)

In [55]:
print(submissions)

[1 1 1 ... 1 1 0]


In [56]:
df_t['Target']=submissions

In [57]:
df_t['ID']=df_test['ID']

In [58]:
df_t.head()

,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,...,H08_Impute_torche,H09_Impute_Charbon de bois,H09_Impute_Gaz,H09_Impute_Ne cuisine pas,H09_Impute_autres,H09_Impute_electricite,H09_Impute_petrole,H09_Impute_traditionnel,Target,ID
0,0.162374,0,1,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,ID_6R2X9XE3RA
1,1.512391,0,1,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,ID_1CQDGXQJW9
2,-1.187643,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,ID_5DQVKM2ODB
3,-0.512634,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,ID_65TQFKG2L2
4,-0.512634,0,0,1,0,0,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,ID_NBG4Q4OAF2


In [59]:
sub=df_t[['ID','Target']]
sub.head()

,ID,Target
0,ID_6R2X9XE3RA,1
1,ID_1CQDGXQJW9,1
2,ID_5DQVKM2ODB,1
3,ID_65TQFKG2L2,0
4,ID_NBG4Q4OAF2,0


In [60]:
sub.to_csv("submission.csv",index=False)